# Fine-tuning

In [ ]:
import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [ ]:
!pip3 install -q -U bitsandbytes
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 8.2 MB/s eta 0:00:00


In [ ]:
# !pip install accelerate
# !pip install transformers --upgrade

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GemmaTokenizer

model_id = "google/gemma-7b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}, token=os.environ['HF_TOKEN'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
text = "Quote: Greed, for lack of a better word,"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: Greed, for lack of a better word, is good. Greed is right. Greed works. Greed clarifies, cuts through, and captures the essence


In [ ]:
# os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from peft import LoraConfig

# Create an instance of the LoraConfig class with specific configurations.
lora_config = LoraConfig(
    # 'r' parameter: This might specify a rate, resolution, or reduction dimension,
    # depending on the context within the model or system it's being applied to.
    # The exact meaning of 'r=8' depends on the internal implementation of LoraConfig.
    r=8,

    # 'target_modules' parameter: Lists the components or modules of the model that
    # the configuration will target. These could refer to different parts of a neural network,
    # especially in the context of an attention mechanism or similar structure, where:
    # - 'q_proj' could refer to the projection for queries,
    # - 'o_proj' for outputs,
    # - 'k_proj' for keys,
    # - 'v_proj' for values,
    # - 'gate_proj', 'up_proj', 'down_proj' could be custom projections or mechanisms
    # within the model, possibly related to gating or hierarchical processing.
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],

    # 'task_type' parameter: Specifies the kind of task the model or system is intended for.
    # 'CAUSAL_LM' indicates a Causal Language Model, a type of model used for generating text
    # where each output token is predicted based on the preceding ones, without looking ahead
    # to future tokens. This is common in generative tasks where understanding the sequence
    # order is crucial.
    task_type="CAUSAL_LM",
)

In [ ]:
from datasets import load_dataset

data = load_dataset("Abirate/english_quotes")
data = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [ ]:
# Display the first 3 examples
for i in range(3):
    print(data['train'][i])

{'quote': '“Be yourself; everyone else is already taken.”', 'author': 'Oscar Wilde', 'tags': ['be-yourself', 'gilbert-perreira', 'honesty', 'inspirational', 'misattributed-oscar-wilde', 'quote-investigator'], 'input_ids': [2, 235366, 2448, 5804, 235289, 4784, 1354, 603, 3303, 3443, 1816], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
{'quote': "“I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.”", 'author': 'Marilyn Monroe', 'tags': ['best', 'life', 'love', 'mistakes', 'out-of-control', 'truth', 'worst'], 'input_ids': [2, 235366, 235285, 235303, 235262, 44166, 235269, 76694, 578, 476, 2356, 93811, 235265, 590, 1501, 23798, 235269, 590, 1144, 921, 576, 2582, 578, 696, 3023, 2811, 577, 6589, 235265, 2005, 1013, 692, 798, 235303, 235251, 6589, 682, 696, 970, 13488, 235269, 1492, 692, 2821, 685, 11944, 1453, 235303, 235251, 24769, 6

In [ ]:
import transformers
from trl import SFTTrainer

def formatting_func(example):
    text = f"Quote: {example['quote'][0]}\nAuthor: {example['author'][0]}"
    return [text]

trainer = SFTTrainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=10,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:451: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
1,1.375600
2,0.487700
3,0.687800
4,0.466700
5,0.246400
6,0.557000
7,0.485600
8,0.172600
9,0.307200
10,0.290400


TrainOutput(global_step=10, training_loss=0.5077055752277374, metrics={'train_runtime': 13.3923, 'train_samples_per_second': 2.987, 'train_steps_per_second': 0.747, 'total_flos': 21555767439360.0, 'train_loss': 0.5077055752277374, 'epoch': 6.67})

In [ ]:
text = "Quote: Greed, for lack of a better word,  "
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=30)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: Greed, for lack of a better word,  is good.
Author: Gordon Gekko
Book: Wall Street
Year: 1987

The movie Wall Street is a 


# Gemma

In [ ]:
# !pip install accelerate
# !pip install transformers --upgrade

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto")

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
input_text = """Summarize the following text in 3 bullet points:


	Opec+ members led by Saudi Arabia and Russia have extended voluntary cuts to oil production for another three months, as they attempt to boost prices that have remained subdued in spite of geopolitical tensions.

The curbs were due to expire at the end of March but will continue until the end of June, according to Saudi Arabia’s state news agency.

The measures add to a series of output cuts by Opec+ members since 2022 designed to support prices amid rising US production and tepid global demand. Since the latest voluntary cuts came into effect in January they have lowered the combined production targets of members by about 2.2mn barrels a day.

“The decision sends a message of cohesion and confirms that the group is not in a hurry to return supply volumes, supporting the view that when this finally happens, it will be gradual,” said Giacomo Romeo, an analyst at Jefferies.

Brent crude, the international benchmark, has risen by 6 per cent and the US equivalent WTI almost 8 per cent since the latest cuts were first announced at the end of November.

But despite tensions in the Middle East, including the Israel-Hamas war and the attacks on commercial shipping by the Houthis, the oil price remains well below the $100 a barrel level last seen in the summer of 2022.

Traders had largely expected the decision to extend the curbs, with crude oil prices rising last week ahead of the announcement. Brent rose more than 2 per cent last week to close above $83 a barrel on Friday, while WTI closed just under $80 a barrel, a rise of more than 4 per cent.

Opec+ was “trying to keep the market in balance”, said Amrita Sen at Energy Aspects. “Oil prices are a lot more stable . . . but they want to ensure the stability continues,” she said.

Saudi Arabia has shouldered most of the curbs, having cut its production by 1mn b/d since July. In total, the kingdom is producing 2mn b/d a less than it did in October 2022. In January, it dropped its plans to expand its daily oil production capacity by 2027 in a major policy reversal.


Summary:
"""
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_length=1000)
print(tokenizer.decode(outputs[0]))

<bos>Summarize the following text in 3 bullet points:


	Opec+ members led by Saudi Arabia and Russia have extended voluntary cuts to oil production for another three months, as they attempt to boost prices that have remained subdued in spite of geopolitical tensions.

The curbs were due to expire at the end of March but will continue until the end of June, according to Saudi Arabia’s state news agency.

The measures add to a series of output cuts by Opec+ members since 2022 designed to support prices amid rising US production and tepid global demand. Since the latest voluntary cuts came into effect in January they have lowered the combined production targets of members by about 2.2mn barrels a day.

“The decision sends a message of cohesion and confirms that the group is not in a hurry to return supply volumes, supporting the view that when this finally happens, it will be gradual,” said Giacomo Romeo, an analyst at Jefferies.

Brent crude, the international benchmark, has risen by 6 

In [ ]:
# Q&A function

def answer_the_question(question):
    input_ids = tokenizer(question, return_tensors="pt").input_ids
    generated_text = model.generate(
        input_ids.to("cuda"),
        max_length=1000,
    )
    answer = tokenizer.decode(generated_text[0], skip_special_tokens=True)
    return answer
question = "Write a short rap about stanford students taking an LLM class."
answer = answer_the_question(question)
print(f"Question: {question}\nAnswer: {answer}")

Question: Write a short rap about stanford students taking an LLM class.
Answer: Write a short rap about stanford students taking an LLM class.

(Verse 1)
Yo, I'm a Stanford student, got my LLM in hand
Taking the AI class, the future's planned
ChatGPT, ChatGPT, the AI flow
Learning the ropes, the knowledge I grow

(Chorus)
LLM, LLM, taking the stage
Generating text, on the digital page
From essays to code, the possibilities are vast
Stanford students, leading the cast

(Verse 2)
Professor's lectures, I'm absorbing the might
Understanding language, day and night
From natural language processing to AI's might
I'm pushing the boundaries, shining so bright

(Chorus)
LLM, LLM, taking the stage
Generating text, on the digital page
From essays to code, the possibilities are vast
Stanford students, leading the cast

(Outro)
So let the knowledge flow, let the AI take hold
We're the future of tech, the story unfolds
From the library to the labs, we're breaking the mold
Stanford students, taking 